# Fetching data from NOMAD

This file is used to collect the perovskite solar cell data for all 43 119 cells from the NOMAD repository. It collects each entry's PCE, ETL, HTL and publication reference. If necessary, more properties can be accessed.

In [1]:
import numpy as np
import pandas as pd
import requests
import pickle

# Fetching data

In [2]:
# Function to extract values from the downloaded entries
def extract_values(entry):
    try:
        htl.append(entry['results']['properties']['optoelectronic']['solar_cell']['hole_transport_layer'])
    except:
        htl.append('None')
    try:
        etl.append(entry['results']['properties']['optoelectronic']['solar_cell']['electron_transport_layer'])
    except:
        etl.append('None')
    try:
        ref.append(entry['references'])
    except:
        ref.append('None')
    return htl, etl, ref

In [3]:
# gets all ~43 119 PSCs from NOMAD

# initialize empty lists where collected values will be stored
htl = []
etl = []
ref = []

page_after_value = None
base_url = 'https://nomad-lab.eu/prod/v1/api/v1/'

# access NOMAD API and query for all cells with the property SolarCell that 
# have information in the mentioned sections:
while True:
    data = requests.post(f'{base_url}entries/query', json={
        "owner": "visible",
        "aggregations": {},
        "query": {
            "and": [
                {"sections:all": ["nomad.datamodel.results.SolarCell"]},
                ]},
        "required": {
            "results":{
                "material": {
                    "chemical_formula_reduced":"*",
                    "structural_type":"*"},
                "properties": {
                   "optoelectronic":{
                      "band_gap":"*",
                      "solar_cell":{
                          "open_circuit_voltage":"*",
                          "short_circuit_current_density":"*",
                          "fill_factor":"*",
                          "efficiency":"*",
                          }}},},
        },
        "pagination": {"page_size": 10,
                       "page_after_value": page_after_value}
        }).json()

    if not data['data']:
        print('debug: no data found')
        break

    # instructions for the last page
    if 'next_page_after_value' not in data['pagination'].keys():
        for entry in data['data']:
            if 'results' not in entry.keys():
                continue
            elif 'chemical_formula_reduced' not in entry['results']['material'].keys():
                continue
            else:
                extract_values(entry)
        break

    page_after_value = data['pagination']['next_page_after_value']

    # extract the values from current page
    for entry in data['data']:
        if 'results' not in entry.keys():
            continue
        else:
            extract_values(entry)

In [ ]:
# put result of query into a pandas dataframe
df_all_ctls = pd.DataFrame({'etl': etl, 'htl': htl, 'ref': ref})

# clean up ref so it shows only the reference to the paper
df_all_ctls['ref'] = df_all_ctls['ref'].apply(lambda x: x[0] if x else None)

# Pickle the result
Fetching all those entries took ca. 30 mins, so I pickle them here.

In [5]:
with open('data_RO1/df_all_ctls.pkl', 'wb') as f:
    pickle.dump(df_all_ctls, f)

# Fetching many properties for all data

In [3]:
base_url = 'https://nomad-lab.eu/prod/v1/api/v1/'

def extract_values(entry):
    try:
         bandgaps.append(entry['results']['properties']['electronic']['band_structure_electronic'][0]['band_gap'][0]['value'])
    except:
         bandgaps.append('None')
    try:
        reduced_formulas.append(entry['results']['material']['chemical_formula_reduced'])
        iupac_formulas.append(entry['results']['material']['chemical_formula_iupac'])
        descriptive_formulas.append(entry['results']['material']['chemical_formula_descriptive'])
    except:
        reduced_formulas.append('None')
        iupac_formulas.append('None')
        descriptive_formulas.append('None')
    try:
        vocs.append(entry['results']['properties']['optoelectronic']['solar_cell']['open_circuit_voltage'])	
    except:
        vocs.append('None')
    try:
        jscs.append(entry['results']['properties']['optoelectronic']['solar_cell']['short_circuit_current_density'])
    except:
        jscs.append('None')
    try:
        ffs.append(entry['results']['properties']['optoelectronic']['solar_cell']['fill_factor'])
    except:
        ffs.append('None')
    try:
        pces.append(entry['results']['properties']['optoelectronic']['solar_cell']['efficiency'])
    except:
        pces.append('None')
    try:
        device_stack.append(entry['results']['properties']['optoelectronic']['solar_cell']['device_stack'])
    except:
        device_stack.append('None')
    try:
        htl.append(entry['results']['properties']['optoelectronic']['solar_cell']['hole_transport_layer'])
    except:
        htl.append('None')
    try:
        etl.append(entry['results']['properties']['optoelectronic']['solar_cell']['electron_transport_layer'])
    except:
        etl.append('None')
    try:
        structural_dimensionality.append(entry['results']['material']['structural_type'])
    except:
        structural_dimensionality.append('None')
    return bandgaps, reduced_formulas, iupac_formulas, descriptive_formulas, vocs, jscs, ffs, pces, device_stack, htl, etl, structural_dimensionality

bandgaps =[]
vocs = []
jscs = []
ffs = []
pces = []
reduced_formulas = []
iupac_formulas = []
descriptive_formulas = []
htl = []
etl = []
device_stack = []
structural_dimensionality = []
page_after_value = None
count = 0

while True:
    data = requests.post(f'{base_url}entries/query', json={
        "owner": "visible",
        "aggregations": {},
        "query": {
            "and": [
                #{"results.material.elements:all": ["Sn"]},
                {"sections:all": ["nomad.datamodel.results.SolarCell"]},
                ]},
        "required": {
            "results":{
                "material": {
                    "chemical_formula_reduced":"*",
                    "structural_type":"*"},
                "properties": {
                   "optoelectronic":{
                      "band_gap":"*",
                      "solar_cell":{
                          "open_circuit_voltage":"*",
                          "short_circuit_current_density":"*",
                          "fill_factor":"*",
                          "efficiency":"*",
                          }}},},
        },
        "pagination": {"page_size": 10,
                       "page_after_value": page_after_value}
        }).json()


    if not data['data']:
        break
    if 'next_page_after_value' not in data['pagination'].keys():
    # make sure to grasp the entries of the last page before breaking
        for entry in data['data']:
            if 'results' not in entry.keys():
                continue
            elif 'chemical_formula_reduced' not in entry['results']['material'].keys():
                continue
            else:
                extract_values(entry)
        break
    page_after_value = data['pagination']['next_page_after_value']

    for entry in data['data']:
        count = count + 1
        print(f"Progress: {count}")
        if 'results' not in entry.keys():
            continue
        #elif 'chemical_formula_reduced' not in entry['results']['material'].keys():
        #    continue
        else:
            extract_values(entry)

Progress: 1
Progress: 2
Progress: 3
Progress: 4
Progress: 5
Progress: 6
Progress: 7
Progress: 8
Progress: 9
Progress: 10
Progress: 11
Progress: 12
Progress: 13
Progress: 14
Progress: 15
Progress: 16
Progress: 17
Progress: 18
Progress: 19
Progress: 20
Progress: 21
Progress: 22
Progress: 23
Progress: 24
Progress: 25
Progress: 26
Progress: 27
Progress: 28
Progress: 29
Progress: 30
Progress: 31
Progress: 32
Progress: 33
Progress: 34
Progress: 35
Progress: 36
Progress: 37
Progress: 38
Progress: 39
Progress: 40
Progress: 41
Progress: 42
Progress: 43
Progress: 44
Progress: 45
Progress: 46
Progress: 47
Progress: 48
Progress: 49
Progress: 50
Progress: 51
Progress: 52
Progress: 53
Progress: 54
Progress: 55
Progress: 56
Progress: 57
Progress: 58
Progress: 59
Progress: 60
Progress: 61
Progress: 62
Progress: 63
Progress: 64
Progress: 65
Progress: 66
Progress: 67
Progress: 68
Progress: 69
Progress: 70
Progress: 71
Progress: 72
Progress: 73
Progress: 74
Progress: 75
Progress: 76
Progress: 77
Progress

In [5]:
df = pd.DataFrame({
    'reduced_formulas': reduced_formulas,
    'iupac_formulas': iupac_formulas,
    'descriptive_formulas': descriptive_formulas,
    'bandgap': bandgaps,
    'voc': vocs,
    'jsc': jscs,
    'ff': ffs,
    'pce': pces,
    'device_stack': device_stack,
    'htl': htl,
    'etl': etl,
    'structural_dimensionality': structural_dimensionality
    })


In [6]:
df.to_csv('pickles/df_all_cells_many_properties.csv', index=False)